# HOMEWORK 2.1: Titanic ML Competition


In [62]:
# Common imports
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn


In [63]:
# to make this notebook's output stable across runs
np.random.seed(42)

## Description of HW2.1: Tackle the Titanic dataset

This is the legendary Titanic ML competition the best of [Kaggle](https://www.kaggle.com/), 
first challenge for you to dive into ML competitions and familiarize yourself with how the Kaggle platform works.
Let's go to the [Titanic challenge](https://www.kaggle.com/c/titanic).

The data is already split into a training set and a test set. However, the test data does *not* contain the labels: your goal is to train the best model you can using the training data, then make your predictions on the test data and REPORT your final score.

The goal is to predict whether or not a passenger survived based on attributes such as their age, sex, passenger class, where they embarked and so on.

Note: Students with the highest score on the assignment will be awarded an additional 10 points as an assignment score. 
All submitted scores will be ranked in descending order and the top 5 students will be awarded an additional +10 points. 

load the data:

In [100]:
train_data = pd.read_csv("/Users/hamody/Documents/ain3001/hw1/train.csv")
test_data = pd.read_csv("/Users/hamody/Documents/ain3001/hw1/test.csv")

Let's take a peek at the top few rows of the training set:

In [65]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The attributes have the following meaning:
* **Survived**: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* **Pclass**: passenger class.
* **Name**, **Sex**, **Age**: self-explanatory
* **SibSp**: how many siblings & spouses of the passenger aboard the Titanic? // Titanik'te yolcunun kaç kardeşi ve eşi var?
* **Parch**: how many children & parents of the passenger aboard the Titanic? // Titanik'te yolcunun kaç çocuğu ve ebeveyni var?
* **Ticket**: ticket id
* **Fare**: price paid (in pounds)
* **Cabin**: passenger's cabin number
* **Embarked**: where the passenger embarked the Titanic

Let's get more info to see how much data is missing:

In [66]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Okay, the **Age**, **Cabin** and **Embarked** attributes are sometimes null (less than 891 non-null),
 especially the **Cabin** (77% are null). 
 We will ignore the **Cabin** for now and focus on the rest. 
 The **Age** attribute has about 19% null values, 
 so we will need to decide what to do with them. 
 Replacing null values with the median age seems reasonable.

The **Name** and **Ticket** attributes may have some value,
 but they will be a bit tricky to convert into useful numbers that a model can consume. 
 So for now, we will ignore them.

Let's take a look at the numerical attributes:

In [67]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* only 38% **Survived**. :(  That's close enough to 40%, so accuracy will be a reasonable metric to evaluate our model.
* The mean **Fare** was £32.20, which does not seem so expensive (but it was probably a lot of money back then).
* The mean **Age** was less than 30 years old.

Let's check that the target is indeed 0 or 1:

In [68]:
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

Now let's take a quick look at all the categorical attributes:

In [69]:
train_data["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [70]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [71]:
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

The Embarked attribute tells us where the passenger embarked: C=Cherbourg, Q=Queenstown, S=Southampton.


## TO-DO 2.1.1: Build your pre-processing pipelines for numerical/categorical attributes.
 - Use SimpleImputer for pre-processing. 
 - Use "Median" Strategy for the SimpleImputer for numerical attributes
 - Use "OrdinalEncoder" function and "most_frequent" strategy for categorical attributes
 - Examine the changes that the simpleimpute function makes to the data and give examples of changed values


In [85]:
# 2.1.1 - build the pipeline for the numerical attributes:

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])




# 2.1.1 - build the pipeline for the categorical attributes:
cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder()),
])





# 2.1.1 - interpret results:
# the missing numerical data of the attributes will be replaced with the median value of it,
# the missing data of the catagorical attributes will be replaced with the most frequent values in these attributes


In [79]:
from sklearn.compose import ColumnTransformer

num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [82]:
X_train = preprocess_pipeline.fit_transform(train_data)
X_train

array([[22.,  1.,  0., ...,  2.,  1.,  2.],
       [38.,  1.,  0., ...,  0.,  0.,  0.],
       [26.,  0.,  0., ...,  2.,  0.,  2.],
       ...,
       [28.,  1.,  2., ...,  2.,  0.,  2.],
       [26.,  0.,  0., ...,  0.,  1.,  0.],
       [32.,  0.,  0., ...,  2.,  1.,  1.]])

The missing values in the numerical attributes is replaced with the median value of the attribute

In the catagorical attributes, the values will be replaced with the most frequent


Let's not forget to get the labels:

In [86]:
y_train = train_data["Survived"]

We are now ready to train a classifier. 

## TO-DO 2.1.2: Use Support a Classifier (SVC, KNN, RandomForest etc.) 
 - Use 3 different Classifier (using sklearn library)
 - Train the selected classifer using "train_data" and "y_train" labels to classify/predict "Survived Passanger" for TEST DATA (test_data) 
 - Use cross-validation method to get avarage accuracy for the dataset 
   (example: 
             from sklearn.model_selection import cross_val_score
             clf1_scores = cross_val_score(clf1, X_train, y_train, cv=10)
             clf1_scores.mean() 
             
 - Show the best prediction accuracy 


In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# 2.1.2 - Classifier 1
Classifier_1 = RandomForestClassifier().fit(X_train, y_train)
clf1_scores = cross_val_score(Classifier_1, X_train, y_train, cv=10)
print(clf1_scores.mean())



# 2.1.2 - Classifier 2

Classifier_2 = SVC().fit(X_train, y_train)

# 2.1.2 - Classifier 3

Classifier_3 = KNeighborsClassifier().fit(X_train, y_train)

# 2.1.2 - Write your the BEST Accuracy



0.8048064918851436


In [117]:
test_d = preprocess_pipeline.fit_transform(test_data)


In [118]:
predictions1 = Classifier_1.predict(test_d)
predictions2 = Classifier_2.predict(test_d)
predictions3 = Classifier_3.predict(test_d)

In [120]:
from sklearn.model_selection import cross_val_score
      clf1_scores = cross_val_score(Classifier_3, X_train, y_train, cv=10)
      clf1_scores.mean()

IndentationError: unexpected indent (665243588.py, line 2)